In [1]:
import os

In [2]:
%pwd

'd:\\mlproject\\Project_MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\mlproject\\Project_MLFlow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [1]:
from projectml.constants import *
from projectml.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile 
from projectml import logger
from projectml.utils.common import _get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {_get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-06 19:29:00,930: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-06 19:29:00,953: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-06 19:29:00,962: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-06 19:29:00,969: INFO: common: created directory at :artifacts]
[2024-02-06 19:29:00,971: INFO: common: created directory at :artifacts/data_ingestion]


[2024-02-06 19:29:03,442: INFO: 2247974253: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 27580
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "287a7d4dbda0737bcee34c832100356a5b6ad338bc958d242280da5f1d289fc5"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2DC2:EA8F6:19533:20370:65C23B25
Accept-Ranges: bytes
Date: Tue, 06 Feb 2024 13:59:02 GMT
Via: 1.1 varnish
X-Served-By: cache-del21750-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1707227942.007440,VS0,VE301
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 1eca2460978b5391b85b9f1109fba4b60285cba8
Expires: Tue, 06 Feb 2024 14:04:02 GMT
Source-Age: 0

]


In [12]:
%pwd

'd:\\mlproject\\Project_MLFlow'